In [6]:
import os
import librosa
import librosa.display
import IPython.display as ipd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### The Medley Instruments Data Set

In [3]:
df = pd.read_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata.csv')

In [4]:
df

,subset,instrument,instrument_id,song_id,uuid4
0,test,clarinet,0,0,0e4371ac-1c6a-51ab-fdb7-f8abd5fbf1a3
1,test,clarinet,0,0,33383119-fd64-59c1-f596-d1a23e8a0eff
2,test,clarinet,0,0,b2b7a288-e169-5642-fced-b509c06b11fc
3,test,clarinet,0,0,151b6ee4-313a-58d9-fbcb-bab73e0d426b
4,test,clarinet,0,0,b43999d1-9b5e-557f-f9bc-1b3759659858
...,...,...,...,...,...
21566,validation,violin,7,226,fe4e8e98-6e0f-5a31-f446-99c10e0ac485
21567,validation,violin,7,226,aa606c78-9ee5-507f-f7e9-67c3530faf0f
21568,validation,violin,7,226,05e15c0a-d530-5f3e-fa82-58c55fa44993
21569,validation,violin,7,226,2dd485de-471d-5d8b-fe92-ef957dac021c


In [5]:
df.nunique()

subset               3
instrument           8
instrument_id        8
song_id            227
uuid4            21571
dtype: int64

In [6]:
df['instrument'].unique()

array(['clarinet', 'distorted electric guitar', 'female singer', 'flute',
       'piano', 'tenor saxophone', 'trumpet', 'violin'], dtype=object)

In [7]:
df.groupby(['instrument']).count()

,subset,instrument_id,song_id,uuid4
instrument,,,,
clarinet,1311,1311,1311,1311
distorted electric guitar,1854,1854,1854,1854
female singer,1744,1744,1744,1744
flute,3555,3555,3555,3555
piano,6032,6032,6032,6032
tenor saxophone,477,477,477,477
trumpet,627,627,627,627
violin,5971,5971,5971,5971


In [8]:
df.groupby(['instrument']).count().sum()

subset           21571
instrument_id    21571
song_id          21571
uuid4            21571
dtype: int64

In [9]:
df['subset'].unique()

array(['test', 'training', 'validation'], dtype=object)

In [10]:
df.groupby(['subset']).count()

,instrument,instrument_id,song_id,uuid4
subset,,,,
test,12236,12236,12236,12236
training,5841,5841,5841,5841
validation,3494,3494,3494,3494


Splitting Dataframe into Train, Validation and Test Dataframes

Saving once:

In [151]:
df.loc[df['subset']=='training']

,subset,instrument,instrument_id,song_id,uuid4
12236,training,clarinet,0,139,163fd2b1-8e98-515a-f501-4742cc6d066f
12237,training,clarinet,0,139,36dcb8eb-da46-5d01-fa9d-0f715c389379
12238,training,clarinet,0,139,9f6d4046-3662-5433-fc44-98c80a428fd8
12239,training,clarinet,0,139,5ef6a0ee-a092-5d49-f64e-a3454e9ab237
12240,training,clarinet,0,139,2048e98b-77b2-5b7d-fe6a-780f399e69fe
...,...,...,...,...,...
18072,training,violin,7,199,284c120e-0986-5220-ff20-a139682268a8
18073,training,violin,7,199,ac044390-8932-5ce7-f3d8-a1a0997f3be5
18074,training,violin,7,199,acfb1f64-e4ed-5422-fbc3-64366eac2fee
18075,training,violin,7,199,af6de7e1-bfee-545b-f114-00d2df94ad84


##### Split Dataframe into train, validation and test dataframes and save

In [ ]:
# Data separated into three folders /training, /validation and /test.
# Dataframe also split into 3 dataframes df_train, df_val, def_test

In [76]:
df.loc[df['subset']=='test'].to_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata_test.csv')

In [77]:
df.loc[df['subset']=='training'].to_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata_training.csv')

In [78]:
df.loc[df['subset']=='validation'].to_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata_validation.csv')

##### Read train, validation and test dataframes

In [7]:
df_train = pd.read_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata_training.csv',index_col=0)

In [8]:
df_val = pd.read_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata_validation.csv',index_col=0)

In [9]:
df_test = pd.read_csv('~/documents/data/audio/medley/csv/Medley-solos-DB_metadata_test.csv',index_col=0)

In [10]:
df_train

,subset,instrument,instrument_id,song_id,uuid4
12236,training,clarinet,0,139,163fd2b1-8e98-515a-f501-4742cc6d066f
12237,training,clarinet,0,139,36dcb8eb-da46-5d01-fa9d-0f715c389379
12238,training,clarinet,0,139,9f6d4046-3662-5433-fc44-98c80a428fd8
12239,training,clarinet,0,139,5ef6a0ee-a092-5d49-f64e-a3454e9ab237
12240,training,clarinet,0,139,2048e98b-77b2-5b7d-fe6a-780f399e69fe
...,...,...,...,...,...
18072,training,violin,7,199,284c120e-0986-5220-ff20-a139682268a8
18073,training,violin,7,199,ac044390-8932-5ce7-f3d8-a1a0997f3be5
18074,training,violin,7,199,acfb1f64-e4ed-5422-fbc3-64366eac2fee
18075,training,violin,7,199,af6de7e1-bfee-545b-f114-00d2df94ad84


In [141]:
df_train['instrument'][500]

'distorted electric guitar'

#### First approach: model for two instruments

In [ ]:
# processing approach:
# define dataframe_subset_instrument
# load into librosa .wav to audio array (n,)
# save arrays with numpy

In [18]:
# working with the Train Set: 404 elements for each instrument

In [19]:
# begin with two instruments

In [147]:
df_train

,pos,subset,instrument,instrument_id,song_id,uuid4
0,12236,training,clarinet,0,139,163fd2b1-8e98-515a-f501-4742cc6d066f
1,12237,training,clarinet,0,139,36dcb8eb-da46-5d01-fa9d-0f715c389379
2,12238,training,clarinet,0,139,9f6d4046-3662-5433-fc44-98c80a428fd8
3,12239,training,clarinet,0,139,5ef6a0ee-a092-5d49-f64e-a3454e9ab237
4,12240,training,clarinet,0,139,2048e98b-77b2-5b7d-fe6a-780f399e69fe
...,...,...,...,...,...,...
5836,18072,training,violin,7,199,284c120e-0986-5220-ff20-a139682268a8
5837,18073,training,violin,7,199,ac044390-8932-5ce7-f3d8-a1a0997f3be5
5838,18074,training,violin,7,199,acfb1f64-e4ed-5422-fbc3-64366eac2fee
5839,18075,training,violin,7,199,af6de7e1-bfee-545b-f114-00d2df94ad84


In [161]:
instruments = df_train['instrument'].unique()

In [111]:
df_train.groupby(['instrument']).count()

,pos,subset,instrument_id,song_id,uuid4
instrument,,,,,
clarinet,251,251,251,251,251
distorted electric guitar,404,404,404,404,404
female singer,318,318,318,318,318
flute,155,155,155,155,155
piano,2401,2401,2401,2401,2401
tenor saxophone,123,123,123,123,123
trumpet,149,149,149,149,149
violin,2040,2040,2040,2040,2040


In [15]:
df_train_clarinet = df_train.loc[df_train['instrument_id']==0]

In [16]:
df_train_clarinet = df_train_clarinet.set_index('subset').reset_index()

In [10]:
df_train_guitar = df_train.loc[df_train['instrument_id']==1]

In [21]:
df_train_guitar = df_train_guitar.set_index('subset').reset_index()

In [11]:
df_train_singer = df_train.loc[df_train['instrument_id']==2]

In [22]:
df_train_singer = df_train_singer.set_index('subset').reset_index()

In [27]:
df_train_singer.head(3)

,subset,instrument,instrument_id,song_id,uuid4
0,training,female singer,2,154,5fe4c1de-b6da-5b50-f4d6-ed9925fc0931
1,training,female singer,2,154,69cbbcff-834b-55c0-f7fe-4d809ebe34e3
2,training,female singer,2,154,94072b83-a4ae-57a9-f196-a74af1e5f3b0


In [24]:
df_train_guitar.shape

(404, 5)

In [25]:
df_train_singer.shape

(318, 5)

In [23]:
# generalize: 3 subset and 8 instruments

In [ ]:
# generalize: 
# subsets: [train, validation, test]
# instruments: [

In [14]:
# we are now working with the Training Data

In [ ]:
# validation subset

In [29]:
# subsets=['val', 'test']
# instruments=['clarinet','guitar','singer']
# instrument_ids=[0,1,2]
# for n, subset in enumerate(subsets):
#     a_{subset} = n
#     #for instrument in instruments:
#      #   df_{subset}_{instrument} = df_{val}.loc[df_{subset}['instrument_id']==0]

In [18]:
df_val.groupby(['instrument']).count()

,subset,instrument_id,song_id,uuid4
instrument,,,,
clarinet,328,328,328,328
distorted electric guitar,495,495,495,495
female singer,284,284,284,284
flute,233,233,233,233
piano,1022,1022,1022,1022
tenor saxophone,29,29,29,29
trumpet,72,72,72,72
violin,1031,1031,1031,1031


In [22]:
df_val_clarinet = df_val.loc[df_val['instrument_id']==0]

In [23]:
df_val_clarinet = df_val_clarinet.set_index('subset').reset_index()

In [32]:
df_val_guitar = df_val.loc[df_val['instrument_id']==1]

In [33]:
df_val_guitar = df_val_guitar.set_index('subset').reset_index()

In [34]:
df_val_singer = df_val.loc[df_val['instrument_id']==2]

In [35]:
df_val_singer = df_val_singer.set_index('subset').reset_index()

In [208]:
# with the test subset

In [20]:
df_test.groupby(['instrument']).count()

,subset,instrument_id,song_id,uuid4
instrument,,,,
clarinet,732,732,732,732
distorted electric guitar,955,955,955,955
female singer,1142,1142,1142,1142
flute,3167,3167,3167,3167
piano,2609,2609,2609,2609
tenor saxophone,325,325,325,325
trumpet,406,406,406,406
violin,2900,2900,2900,2900


In [21]:
df_test_clarinet = df_test.loc[df_test['instrument_id']==0]

In [8]:
df_test_clarinet = df_test_clarinet.set_index('subset').reset_index()

In [10]:
df_test_guitar = df_test.loc[df_test['instrument_id']==1]

In [11]:
df_test_guitar = df_test_guitar.set_index('subset').reset_index()

In [12]:
df_test_singer = df_test.loc[df_test['instrument_id']==2]

In [13]:
df_test_singer = df_test_singer.set_index('subset').reset_index()

### Loading .wav audio files and saving them to .npy numpy array files

In [26]:
filename_pre = "Medley-solos-DB"

In [18]:
def load_files(df):
    """loads audio files into a list with librosa
    input: dataframe of a single subset and instrument
    for example: training set, distorted electric guitar
    output: list of amplitude vectors"""
    amplitude_df=[]
    subset = df['subset'][0]
    instrument_id = df['instrument_id'][0]
    folderpath = "../../../../../../documents/data/audio/medley/"+subset+"/"
    for file_id in df['uuid4']:
        filepath = folderpath+filename_pre+'_'+subset+'-'+str(instrument_id)+'_'+file_id+'.wav'
        amplitude_1, sr = librosa.load(filepath)
        amplitude_df.append(amplitude_1)
    return amplitude_df

In [146]:
# loading audio files with librosa into lists

In [173]:
# run this cells only whe loading files for the first time
# otherwise load amplitude vectors from saved .npy files 

In [52]:
amplitude_train_clarinet = load_files(df_train_clarinet)

In [53]:
amplitude_train_guitar = load_files(df_train_guitar)

In [234]:
amplitude_train_singer = load_files(df_train_singer)

In [19]:
def save_amp_list(list_amplitudes,filename):
    """saves the amplitude vectors in a list into a file
    input: list of amplitude vectors, name of the file where to save
    for example: amp_tr_clarinet to indicate subset and instrument
    """
    folderpath="../../../../../../documents/data/audio/medley/audio_arrays/"
    with open (folderpath+filename, 'wb') as f:
        for amplitude in list_amplitudes:
            np.save(f,amplitude)

In [211]:
# saving amplitude lists to numpy array file .npy

In [224]:
save_amp_list(amplitude_train_clarinet,'amp_tr_clarinet')

In [213]:
save_amp_list(amplitude_train_guitar,'amp_tr_guitar')

In [235]:
save_amp_list(amplitude_train_singer,'amp_tr_singer')

In [20]:
def load_amp_file(filename,df):
    """input: filename, for example: amp_tr_clarinet
    output: list of amplitude vectors
    """
    # load amplitude vector
    # the corresponding dataframe must be at hand
    n_elements=(len(df))
    folderpath="../../../../../../documents/data/audio/medley/audio_arrays/"
    amp_list=[]
    with open (folderpath+filename, 'rb') as f:
        for n in range(n_elements):
            amp_1 = np.load(f)
            amp_list.append(amp_1)
    return amp_list

In [219]:
amp_tr_clarinet = load_amp_file('amp_tr_clarinet', df_train_clarinet)

In [220]:
amp_tr_guitar = load_amp_file('amp_tr_guitar', df_train_guitar)

In [236]:
amp_tr_singer = load_amp_file('amp_tr_singer', df_train_singer)

In [159]:
# it works!

In [ ]:
# with the val dataframes

In [227]:
amplitude_val_clarinet = load_files(df_val_clarinet)

In [230]:
amplitude_val_guitar = load_files(df_val_guitar)

In [207]:
amplitude_val_singer = load_files(df_val_singer)

In [14]:
#save validation subset amplitude arrays

In [228]:
save_amp_list(amplitude_val_clarinet,'amp_val_clarinet')

In [231]:
save_amp_list(amplitude_val_guitar,'amp_val_guitar')

In [232]:
save_amp_list(amplitude_val_singer,'amp_val_singer')

In [ ]:
#load validation subset amplitude arrays

In [24]:
amp_val_clarinet = load_amp_file('amp_val_clarinet', df_val_clarinet)

In [239]:
amp_val_guitar = load_amp_file('amp_val_guitar', df_val_guitar)

In [240]:
amp_val_singer = load_amp_file('amp_val_singer', df_val_singer)

In [ ]:
#with the test dataframes

In [27]:
amplitude_test_clarinet = load_files(df_test_clarinet)

In [30]:
amplitude_test_guitar = load_files(df_test_guitar)

In [33]:
amplitude_test_singer = load_files(df_test_singer)

In [37]:
#save test subset amplitude arrays

In [29]:
save_amp_list(amplitude_test_clarinet,'amp_test_clarinet')

In [31]:
save_amp_list(amplitude_test_guitar,'amp_test_guitar')

In [34]:
save_amp_list(amplitude_test_singer,'amp_test_singer')

In [38]:
#load test subset amplitude arrays

In [39]:
amp_test_clarinet = load_amp_file('amp_test_clarinet', df_test_clarinet)

In [40]:
amp_test_guitar = load_amp_file('amp_test_guitar', df_test_guitar)

In [41]:
amp_test_singer = load_amp_file('amp_test_singer', df_test_singer)

In [ ]:
#benchmark

In [42]:
df_test.groupby(['instrument']).count()

,subset,instrument_id,song_id,uuid4
instrument,,,,
clarinet,732,732,732,732
distorted electric guitar,955,955,955,955
female singer,1142,1142,1142,1142
flute,3167,3167,3167,3167
piano,2609,2609,2609,2609
tenor saxophone,325,325,325,325
trumpet,406,406,406,406
violin,2900,2900,2900,2900


In [44]:
len(amp_test_clarinet), len(amp_test_guitar), len(amp_test_singer)

(732, 955, 1142)

In [45]:
trues=[]
for i in range(len(amp_test_clarinet)):
    trues.append((amp_test_clarinet[0] == amplitude_test_clarinet[0]).all())

In [48]:
trues=[]
for i in range(len(amp_test_guitar)):
    trues.append((amp_test_guitar[0] == amplitude_test_guitar[0]).all())

In [52]:
trues=[]
for i in range(len(amp_test_singer)):
    trues.append((amp_test_singer[0] == amplitude_test_singer[0]).all())

In [53]:
df_trues=pd.DataFrame(trues)

In [54]:
df_trues.describe()

,0
count,1142
unique,1
top,True
freq,1142


In [55]:
# it worked!

In [2]:
# up here: second part, load data and ML in different notebook
# intrument_recognition.ipynb

In [1]:
#up here can be deleted, is implemented in instrument_recognition.ipynb